In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. We run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this project, we only look at GDP data from the first quarter of 2000 onward.

In [3]:
# We use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

allcities = pd.read_csv('City_Zhvi_AllHomes.csv')

df1 = pd.DataFrame(list(states.items()), columns= ['State Code', 'State Name'])

cities = pd.merge(allcities, df1, how = 'left', left_on = 'State', right_on= 'State Code')

cities.drop(cities.columns[6:51], axis=1, inplace=True)
cities.drop(cities.columns[[206]], axis=1, inplace=True)
cities.set_index(['State Name','RegionName'], inplace= True)

cities

,,RegionID,State,Metro,CountyName,SizeRank,2000-01,2000-02,2000-03,2000-04,2000-05,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State Name,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,6181,NY,New York,Queens,1,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
California,Los Angeles,12447,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,204400.0,207000.0,209800.0,212300.0,214500.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
Illinois,Chicago,17426,IL,Chicago,Cook,3,136800.0,138300.0,140100.0,141900.0,143700.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
Pennsylvania,Philadelphia,13271,PA,Philadelphia,Philadelphia,4,52700.0,53100.0,53200.0,53400.0,53700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
Arizona,Phoenix,40326,AZ,Phoenix,Maricopa,5,111000.0,111700.0,112800.0,113700.0,114300.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,398292,WI,Green Bay,Brown,10726,101100.0,101500.0,102700.0,103700.0,105200.0,...,149900,150100,150300,150000,149200,149900,151400,152500,154100,155900
New York,Urbana,398343,NY,Corning,Steuben,10727,77600.0,79400.0,80600.0,80200.0,81000.0,...,135700,136400,137700,138700,140500,143600,145000,144000,143000,143000
Wisconsin,New Denmark,398496,WI,Green Bay,Brown,10728,113900.0,114000.0,115800.0,117600.0,119100.0,...,188700,189800,190800,191200,191200,191700,192800,194000,196300,198900


In [4]:
def get_list_of_university_towns():
    
    lst = pd.read_fwf('university_towns.txt', header = None)
    

    unitowns= []
    state=''
    region=''
    
    
    for name in lst[0]:
        if name.find('[ed') > 0 :
                state = name
        else:

                region = name
                
                unitowns.append([state,region])
        
    unitow=pd.DataFrame(unitowns, columns = ['State', 'RegionName'])
 
    unitow['State'] = unitow['State'].str.replace(r"\[.*\]","").str.rstrip()
     
   
    unitow['RegionName'] = unitow['RegionName'].str.split("\s+\(").str[0]

#     '''Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list.
    
#     The following cleaning was done:

#     1. For "State", removing characters from "[" to the end.
#     2. For "RegionName", when applicable, removing every character from " (" to the end.
      
    
    return unitow

get_list_of_university_towns()   

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [5]:
gdp = pd.read_excel('gdplev.xls',header = None, skiprows = 220, usecols = [4,6], names =['Quarter','GDP'])
gdp = gdp.set_index('Quarter')
gdp['change'] =  gdp['GDP'] - gdp['GDP'].shift(periods =1)
gdp['change'].iloc[0] = 0
gdp['RecessionStart'] = (gdp['change'] < 0) & (gdp['change'].shift(-1) < 0)

def get_recession_start():
#     '''Returns the year and quarter of the recession start time as a 
#     string value in a format such as 2005q3'''
    
    return gdp['RecessionStart'].idxmax()

get_recession_start()

'2008q3'

In [6]:
gdp['RecessionEnd'] = (gdp['change'] > 0) & (gdp['change'].shift(1) > 0) & (gdp['change'].shift(2) < 0) & (gdp['change'].shift(3) < 0)
def get_recession_end():
    
#     '''Returns the year and quarter of the recession end time as a 
#     string value in a format such as 2005q3'''
       
    return gdp['RecessionEnd'].idxmax()
get_recession_end()

'2009q4'

In [7]:
gdp['RecessionBottom'] = (gdp['RecessionEnd']== False) & (gdp['RecessionEnd'].shift(-2)== True)
def get_recession_bottom():
#     '''Returns the year and quarter of the recession bottom time as a 
#     string value in a format such as 2005q3'''
    
    return gdp['RecessionBottom'].idxmax()
get_recession_bottom()

'2009q2'

In [9]:
housing = cities[cities.columns[5:206]]

housing.columns = pd.to_datetime(housing.columns)

housing = housing.resample('Q', axis=1).mean()
housing = housing.rename(columns=lambda x: str(x.to_period('Q')).lower())
def convert_housing_data_to_quarters():
#     '''Converts the housing data to quarters and returns it as mean 
#     values in a dataframe.
#     '''
    
    return housing
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State Name,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [37]:
def run_ttest():
    housing_sliced = housing[['2008q2','2008q3','2008q4','2009q1','2009q2']]
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    ul = get_list_of_university_towns()
    
    housing_sliced['PriceRatio'] = housing_sliced['2008q2'].div(housing_sliced[rec_bottom])
    
    univlist = ul.to_records(index= False).tolist()
    group1 = housing_sliced.loc[univlist].dropna(how='all')
    group2 = housing_sliced.loc[~housing_sliced.index.isin(univlist)]
    
    (t,p) = ttest_ind(group1['PriceRatio'], group2['PriceRatio'], nan_policy='omit')
    
    if p<0.1:
        different = True
    else:
        different = False
        
    if group1['PriceRatio'].mean() > group2['PriceRatio'].mean():
        better = "non-university town"
    else:
        better = "university town"
    
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Returns the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p is 
    the exact p value returned from scipy.stats.ttest_ind(). The
    value for better is either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return different, p, better

run_ttest()

C:\Users\vagis\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\vagis\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  # Remove the CWD from sys.path while we load stuff.


(True, 0.002724063704753125, 'university town')

The p value is 0.002 which is smaller than the significance threshold. Thus, we reject the null hypothesis.